##DYNA : Integrating Planning, Acting and Learning

###Enviroment

For this assignment I selected the Frozen Lake environment from OpenAi gym. The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction, i.e., the environment is stochastic. The agent recieves a reward of 1 if if reaches the goal and 0 otherwise.

'Solving' this environment is defined as getting average reward of 0.78 over 100 consecutive trials.



In [1]:
import gym
import numpy as np

# slipperly lake, so actions not deterministic!
env = gym.make('FrozenLake-v0')

env.render()

next_s, r, is_end, inf = env.step(action=0)
print(next_s, r, is_end, inf)
env.render()

INFO:gym.envs.registration:Making new env: FrozenLake-v0
[2017-02-26 22:39:51,511] Making new env: FrozenLake-v0


SFFF
FHFH
FFFH
HFFG

(4, 0.0, False, {'prob': 0.3333333333333333})
SFFF
FHFH
FFFH
HFFG
  (Left)


I first show that simple standard direct RL methods like Q-learning and SARSA are able to solve this task. In order to make the task more interesting wemodify the reward structure. In order to encourage the RL agent to reach the goal faster, a reward of -1 is added for every timestep that passes without reaching the goal. The reward for reaching the goal is amplified from 1 to 100.


Next we analyse the performance of expected SARSA and double-Q learning on this problem. Finally we see if adding planning capabilities is able to improve the agents performance.
It is worth noting that intuitively one would not expect planning to help. This is because the environment is stochastic

###Q-Learning

In [2]:
import qlearning

In [3]:
env = gym.make("FrozenLake-v0")
Q=qlearning.train(env,planning=False)

INFO:gym.envs.registration:Making new env: FrozenLake-v0
[2017-02-26 22:40:00,119] Making new env: FrozenLake-v0


Evaluate the learned Q-table for 1000 episodes

In [4]:
env = gym.make("FrozenLake-v0")
qlearning.evaluate(env,Q)

INFO:gym.envs.registration:Making new env: FrozenLake-v0
[2017-02-26 22:40:14,800] Making new env: FrozenLake-v0


1000 episodes finished in an average of 48.1456298828. Running score: 0.81


###SARSA

The performance of SARSA is very similar to Q-learning, perhaps slightly better. On average SARSA took more steps to reach the goal than Q-learning over multiple runs

In [5]:
import sarsa

In [6]:
Q = sarsa.train(env,planning=False)

In [7]:
sarsa.evaluate(env,Q)

1000 episodes finished in an average of 46.1297912598. Running score: 0.79


We see after 10000 episondes the running average reward is 0.8 (approx) at which point this environment may be considered solved. Thus q-learning is able to perform quite well on this task. In order to evaluate the policy being learned, we play out 1000 episodes using the learned Q-table.

###Planning

Planning can typically be used in order to improve the policy by combining direct RL-methods or learning methods with simulated rather than real experience. This involves a model learning stage. The simplest type of model is deterministic. Here we experiment with DYNA-Q based planning agensts. Note that as the environment is stochastic, using a determistic model can be expected to hurt performance, as we show through experimentation. 

For all the experiments in this notebook we used 15 planning steps. We experimented with a larger number of steps, however the gains were marginal as compared to the increased computational complexity.

In [11]:
def plan(Q,lr,y,model_nextS,model_nextR,plan_steps=15):
    
    vis_before = np.nonzero(model_nextS)
    pstates = list(set(vis_before[0]))

    psteps=0        
    
    while psteps<=plan_steps-1:

        #select a state at random from previous states
        if pstates:
            rstate = random.sample(pstates,1)
            rstate = rstate[0]
        else:
            rstate=0
        #find previously sampled actions
        pacts = np.nonzero(model_nextS[rstate,:])
        if pacts[0].any():
            ract = random.sample(pacts[0],1)
            ract = ract[0]
        else:
            ract=0

        #simulate experience from model
        r_hat = (model_nextR[rstate,ract]).astype('int32')
        s_hat = (model_nextS[rstate,ract]).astype('int32')
    
        #use Q-learning for planning
        Q[rstate,ract] = Q[rstate,ract] + lr*(r_hat + y*np.max(Q[s_hat,:]) - Q[rstate,ract])
        psteps+=1

    return Q   

We now traing Q-learning with 15 planning steps, that also uses Q-learning on simulated experience.

In [12]:
env = gym.make("FrozenLake-v0")
Q=qlearning.train(env,planning=True)

INFO:gym.envs.registration:Making new env: FrozenLake-v0
[2017-02-26 22:43:56,564] Making new env: FrozenLake-v0


In [13]:
qlearning.evaluate(env,Q)

1000 episodes finished in an average of 22.0339813232. Running score: 0.19


As expected, we see that the planning agent doesn't perform as well as SARSA or Q-learning in terms of the average achieved reward. Interestingly, althought the DYNA agent reaches the goal less often it does so in a much smaller number of steps. We observe this trend over multiple runs of the algorithm.

In order to better understand how the stochasticity of the environment affects learning and planning agents, we also attempt to solve the FrozenLake-v0 environment using expected SARSA and Double Q-learning. Intuitively expected SARSA should also suffer due to the enviromental stochasticity.

### Expected SARSA

In [14]:
import esarsa

In [15]:
#env = gym.make("FrozenLake-v0")
Q = esarsa.train(env,planning=False)

In [16]:
esarsa.evaluate(env,Q)

1000 episodes finished in an average of 23.2409629822. Running score: 0.31


We see that expected SARSA does not perform as well as SARSA or Q-learning in terms of average reward. Like planning agents, expected sarsa also reaches the goal faster than SARSA and Q-learning (averaged of multiple 1000 episode runs)

### Double Q-learning

Double Q-learning was proposed to aleviate the positve bias induced by the max-operation in standard Q-learning.

In [17]:
import doubleQ

In [18]:
Q = doubleQ.train(env)

In [19]:
doubleQ.evaluate(env,Q)

1000 episodes finished in an average of 47.3909873962. Running score: 0.85


We see that double-Q learning is also able to solve this task, however performance is very similar to both SARSA and Q-learning

### Changing the Reward structure

We have seen that Planning agents and more elaborate methods like expected SARSA are not able to perform well on this task, as compared to Q-learning and SARSA. We believe that one of the reasons for this the stochasticity of the environment.

An interesting feature of both DYNA-Q and Expected SARSA is that they find solutions in far fewer timesteps. In order to see if planning would help when the agent is trying to reach the goal as fast as possible. Accordingly, we modify the reward structure of the environment such that a reward of -1 is received for every step not reaching the goal, and a reward of 100 is received for reaching the goal.

### Q-learning

In [20]:
import qlearning_v2

In [21]:
qlearning_v2.train(env,planning=False)

array([[  0.94333024,  35.5245364 ,   4.63881496,   4.98135444],
       [  4.59028076,   5.75362184,   0.81      ,  29.25229994],
       [  3.81604308,  28.73416765,   8.98372291,   0.96211783],
       [  0.81      ,   3.71706294,   0.80515213,  26.6912042 ],
       [ 39.67844756,   0.8991    ,  17.24072347,   4.91770911],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [ 32.66565311,   0.72695224,   8.57841777,   0.81      ],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [  0.89357637,   7.34815066,   4.091276  ,  46.1431974 ],
       [  0.89743387,  53.97643532,   0.80919   ,   0.9       ],
       [ 58.16379584,   0.9       ,   0.9       ,   0.89788736],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [  8.33638184,   0.8991    ,  68.32522591,   6.32096236],
       [  0.999     ,  86.79120795,   1.        ,   1.        ],
       [  1.        ,   1

In [22]:
qlearning_v2.evaluate(env,Q)

1000 episodes finished in an average of 47.9901123047. Running score: 32.65


###SARSA

In [23]:
import sarsa_v2

In [24]:
sarsa_v2.train(env,planning=False)

array([[ 34.75528213,  17.28041267,  17.79772388,  19.47401936],
       [  0.89829   ,   0.9       ,   0.89658333,  18.05835613],
       [  0.97953514,   0.98061275,   0.9798291 ,  11.68339026],
       [  0.8991    ,   0.9       ,   0.80919   ,  10.11074038],
       [ 35.91848636,  16.16123361,  14.90787948,  13.49333941],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [  0.97854486,  10.28487568,   0.80702281,   1.70514021],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [ 18.20477985,   7.5667489 ,  18.80996821,  39.72158818],
       [  5.88082956,  42.98908227,   9.17575702,   5.12439196],
       [  3.82441493,   0.9       ,  22.9392794 ,   0.8991    ],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [  1.        ,   1.        ,   1.        ,   1.        ],
       [ 12.95221147,  12.96386082,  58.05361418,   3.68938596],
       [  1.75168463,  83.06348376,   5.93289031,   4.41166611],
       [  1.        ,   1

In [25]:
sarsa_v2.evaluate(env,Q)

1000 episodes finished in an average of 47.6024856567. Running score: 26.85


### DYNA-Q

In [26]:
qlearning_v2.train(env,planning=True)

array([[  7.74093828e+00,   9.36865440e-03,   3.88936870e-04,
          3.67106169e-01],
       [  3.88936870e-04,   3.88936870e-01,   1.00000000e-09,
          1.00000000e-09],
       [  1.00000000e-09,   1.00000000e-06,   1.00000000e-06,
          1.00000000e-06],
       [  1.00000000e-03,   1.00000000e-06,   1.00000000e-03,
          1.00000000e-06],
       [  9.52099127e+00,   9.34904859e-03,   1.00000000e-03,
          2.70027415e-01],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00],
       [  2.83471752e-02,   1.00000000e-03,   1.00000000e-03,
          1.00000000e-03],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00],
       [  1.00000000e-03,   1.73044608e-02,   1.00000000e-03,
          1.13277660e+01],
       [  1.00000000e-03,   2.00009661e+01,   1.00000000e-03,
          1.00000000e-03],
       [  1.73312388e-02,   2.79560021e+01,   1.00000000e-03,
          1.00000000e-03],
       [  1.00000000e

In [27]:
qlearning_v2.evaluate(env,Q)

1000 episodes finished in an average of 48.1672592163. Running score: 27.92


In the case where the reward is maxized if the agent reaches the fast as possible, we see that the planning does atleast as well as Q-learning as SARSA. This is true even though the environment is stochastic. The planning agent now takes almost as long as Q-learning and SARSA. Over several runs we report that the planning agent performs slightly better than the direct RL methods in terms of average reward.